## importing required Libraries

In [44]:
import warnings
warnings.filterwarnings('ignore')
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import string
import re
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from collections import Counter
import nltk
nltk.download('stopwords')
vocab = Counter()



[nltk_data] Downloading package stopwords to /home/neel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### stop words in english to remove them from corpus

In [45]:
stop_words = set(stopwords.words('english'))

### Helper Functions to read Data from Files

In [46]:
dir1 = 'review_polarity/txt_sentoken/neg'
dir2 = 'review_polarity/txt_sentoken/pos'

In [47]:
text = []
tar =[]
def open_doc(file_name,target):
    file = open(file_name)
    data = file.read()
    file.close()
    text.append(data)
    tar.append(target)

In [48]:
count=0
for file_name in listdir(dir1):
    if not file_name.endswith('.txt'):
        continue
    open_doc(dir1+'/'+file_name,0)
    count+=1
    
    
for file_name in listdir(dir2):
    if not file_name.endswith('.txt'):
        continue
    open_doc(dir2+'/'+file_name,1)
    count+=1

In [49]:
count


2000

### A little bit of data preparation 

In [50]:
d ={'Text':text,'Target':tar}

In [51]:
train = pd.DataFrame(data=d)

In [52]:
train['Text']=train['Text'].map(lambda x : x.split())

In [53]:
train['Text'] = train['Text'].map(lambda x:  [word for word in x if not word.startswith(('http','@'))])
re_punc = re.compile('[%s]'%re.escape(string.punctuation))
train['Text'] = train['Text'].map(lambda x:  [re_punc.sub('',w) for w in x])
train['Text'] = train['Text'].map(lambda x: [word for word in x if word.isalpha()])
train['Text'] = train['Text'].map(lambda x: [w.lower() for w in x])    
train['Text'] = train['Text'].map(lambda x:  [w for w in x if not w in stop_words ]) 
train['Text'] = train['Text'].map(lambda x: [w for w in x if len(w)>1])
#train['Text'] = train['Text'].map(lambda x: [lemmatizer.lemmatize(w) for w in x ]) #### Sometimes lemmatization is not just a good idea

In [54]:
train['Text']

0       [adam, sandler, turns, charm, latest, romantic...
1       [dont, need, seen, original, species, apprecia...
2       [production, early, days, initial, aborted, pr...
3       [retelling, classic, story, joan, arc, popular...
4       [capsule, godawful, comedy, thats, amazingly, ...
                              ...                        
1995    [time, bandits, director, terry, gilliam, diff...
1996    [scream, like, predecessor, genrecrossing, fil...
1997    [wednesday, march, murder, investigation, unde...
1998    [times, success, particular, film, depends, en...
1999    [farrelly, brothers, third, film, dumb, dumber...
Name: Text, Length: 2000, dtype: object

In [55]:
#from sklearn.feature_extraction.text import TfidfVectorizer
#vectorizer = TfidfVectorizer()
#vectorizer.fit_transform(text)
#len(vectorizer.get_feature_names())

In [56]:
for line in train['Text']:
    vocab.update(line)



In [57]:
len(vocab)

46556

In [58]:
new_voc = vocab.copy()
for key,val in vocab.items():
    if val<5:
        del new_voc[key]



In [59]:
o = list(new_voc)

### TF-IDF algorithm from scratch

In [ ]:
count =0
arr = []
for line in train['Text']:
    vec = np.zeros(len(new_voc))
    for word in line:
        
        if new_voc[word]!=0:
            vec[o.index(word)]=(line.count(word)/len(line))*np.log(len(train)/new_voc[word])
    arr.append(vec)
    count+=1
        #df.append(list(vec))
df  = np.array(arr)

In [ ]:
df.shape

In [ ]:
Xtrain,Xtest,ytrain,ytest = train_test_split(df,train['Target'],test_size = 0.3,random_state = 1)

In [ ]:
ytrain.shape

#### Simple Logistic Regression

In [ ]:

lr = LogisticRegression()
lr.fit(Xtrain,ytrain)
pre = lr.predict(Xtest)
acc =accuracy_score(ytest,pre)
print(acc*100)

80% accuracy using simple logistic regression, we can do better than this !!!

#### ANN model 

In [ ]:
model = Sequential()
model.add(Dense(64,input_shape = (Xtrain.shape[1],),activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))



In [ ]:
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics =['accuracy'])

In [ ]:
model.fit(Xtrain,ytrain,epochs=10)

In [ ]:
loss, acc = model.evaluate(Xtest, ytest, verbose=1)
print( ' Test Accuracy: %f ' % (acc*100))

We did better than simple logistic regression but training accuracy is 100% and accuaracy on test data is 84% Clearly, this model is overfitting we'll improve it in next commit